<a href="https://colab.research.google.com/github/biodatlab/nbdt-llm/blob/main/Build_Abst_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Builds a database of vector embeddings from list of abstracts

## Some Setup

In [ ]:
!pip install numpy==1.23.2
!pip install transformers==4.28.0
!pip install -U sentence-transformers
!pip install datasets
!pip install langchain
!pip install torch
!pip install -U spacy
!python -m spacy download en_core_web_sm
!pip install tensorflow
!pip install openai
!pip install -qU pinecone-client[grpc]
!pip install Cython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 50.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=9aa8416b18fcdbae784bd197d505fc6117786b3d4d0c0eecabcb5fea06179edb
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
Looking in indexes: https://pypi.

In [ ]:
!gdown 1-JiQjBZXUNsFoR0SQ8CQTZTKJMM1bsys
!gdown 1--NNLlqTWx6zSRdhDN_yjPrht4vEhHVH
!gdown 1NfMFVYC9MFFF1brutfvjUsoqC93kD_Li

Downloading...
From: https://drive.google.com/uc?id=19fuIGZBGFA_OqkVh_VdjVsvM9tZs1-gC
To: /content/covid_ta2.csv
100% 394k/394k [00:00<00:00, 155MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-JiQjBZXUNsFoR0SQ8CQTZTKJMM1bsys
To: /content/bioarxiv.csv
100% 54.3M/54.3M [00:00<00:00, 217MB/s]
Downloading...
From: https://drive.google.com/uc?id=1--NNLlqTWx6zSRdhDN_yjPrht4vEhHVH
To: /content/PLOS_ONE2.csv
100% 57.6M/57.6M [00:00<00:00, 237MB/s]
Downloading...
From: https://drive.google.com/uc?id=1NfMFVYC9MFFF1brutfvjUsoqC93kD_Li
To: /content/arxiv2.csv
100% 5.59M/5.59M [00:00<00:00, 98.6MB/s]


In [ ]:
import os
import shutil

import numpy as np
import pandas as pd
# import openai
import pinecone
import spacy
from tqdm.auto import tqdm

nlp = spacy.load("en_core_web_sm")

# openai.api_key = ''
PINECONE_API_KEY = 'c590f8d4-d335-4ccc-9743-fd90ab8905b8'
PINECONE_ENV = 'us-west4-gcp-free'

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
    )

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
index_name = "reviewer-assignment"

if index_name not in pinecone.list_indexes():
  pinecone.create_index(
      name=index_name,
      dimension=768,
      metric='cosine'
  )
# pinecone.create_index(
#       name=index_name,
#       dimension=model.config.hidden_size,
#       metric='cosine'
#   )
index = pinecone.GRPCIndex(index_name)

In [ ]:
# from tqdm.auto import tqdm

# batch_size = 128

# for i in tqdm(range(0, len(bioarx), batch_size)):
#     # find end of batch
#     i_end = min(i+batch_size, len(bioarx))
#     # create IDs batch
#     ids = [str(x) for x in range(i, i_end)]
#     # create metadata batch
#     metadatas = [{'abstract': row['abstract'],'identifier':row['identifier']} for row in bioarx[i:i_end]]
#     # create embeddings
#     em = [get_embeddings_lbert(row['abstract'],sents_encoder).tolist()[0] for row in bioarx[i:i_end]]
#     # create records list for upsert
#     records = zip(ids, em, metadatas)
#     # upsert to Pinecone
#     index.upsert(vectors=records)

# # check number of records in the index
# index.describe_index_stats()

  0%|          | 0/226 [00:00<?, ?it/s]

{'dimension': 768,
 'index_fullness': 0.2,
 'namespaces': {'': {'vector_count': 28852}},
 'total_vector_count': 28852}

In [ ]:
from transformers import LongformerTokenizer, LongformerModel
import torch

def create_lbert_embed(sents,bundle):
  tokenizer = bundle[0]
  model = bundle[1]
  model.cuda()
  tokens = tokenizer(sents,padding=True,truncation=True,return_tensors='pt')
  device = torch.device('cuda')
  tokens = tokens.to(device)
  with torch.no_grad():
    embeds = model(**tokens, output_hidden_states=True,return_dict=True).pooler_output
  return embeds.cpu()

In [ ]:
def upsert(data,index,create_embed,bundle,name=''):
  batch_size = 128
  """
  data : list of rows with an 'abstract' and an 'identifier' field
  index : pinecone Index object
  create_embed : function that creates the embedding given an abstract
  """
  for i in tqdm(range(0, len(data), batch_size)):
      # find end of batch
      i_end = min(i+batch_size, len(data))
      # create IDs batch
      ids = [name + '-' + str(x) for x in range(i, i_end)]
      # create metadata batch
      metadatas = [{'abstract': row['abstract'],'identifier':row['identifier']} for row in data[i:i_end]]
      # create embeddings
      em = [create_embed(row['abstract'],bundle).tolist()[0] for row in data[i:i_end]]
      # create records list for upsert
      records = zip(ids, em, metadatas)
      # upsert to Pinecone
      index.upsert(vectors=records)

  # check number of records in the index
  index.describe_index_stats()

In [ ]:
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")
model = LongformerModel.from_pretrained("allenai/longformer-base-4096")
lbert_bundle = (tokenizer,model)

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
bioarx = pd.read_csv('bioarxiv.csv')
bioarx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28852 entries, 0 to 28851
Data columns (total 9 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   title                             28852 non-null  object
 1   authors                           28852 non-null  object
 2   author_corresponding              28852 non-null  object
 3   author_corresponding_institution  28851 non-null  object
 4   date                              28852 non-null  object
 5   type                              28852 non-null  object
 6   category                          28852 non-null  object
 7   abstract                          28852 non-null  object
 8   published                         17018 non-null  object
dtypes: object(9)
memory usage: 2.0+ MB


In [ ]:
bioarx = bioarx[['title','abstract']]
bioarx = bioarx.rename(columns={'title':'identifier'})
bioarx = bioarx.to_dict('records')

In [ ]:
plos = pd.read_csv('PLOS_ONE2.csv')
plos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17048 entries, 0 to 17047
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   level_0   17048 non-null  int64 
 1   Title     17048 non-null  object
 2   Author    17048 non-null  object
 3   Abstract  17048 non-null  object
 4   Journal   17048 non-null  object
 5   Subject   17048 non-null  object
dtypes: int64(1), object(5)
memory usage: 799.2+ KB


In [ ]:
plos = plos[['Title','Abstract']]
plos = plos.rename(columns={'Title':'identifier','Abstract':'abstract'})
plos = plos.to_dict('records')

In [ ]:
arx = pd.read_csv('arxiv2.csv')
arx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3372 entries, 0 to 3371
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              3372 non-null   object
 1   submitter       3372 non-null   object
 2   authors         3372 non-null   object
 3   title           3372 non-null   object
 4   journal-ref     623 non-null    object
 5   categories      3372 non-null   object
 6   abstract        3372 non-null   object
 7   versions        3372 non-null   object
 8   update_date     3372 non-null   object
 9   authors_parsed  3372 non-null   object
dtypes: object(10)
memory usage: 263.6+ KB


In [ ]:
arx = arx[['title','abstract']]
arx = arx.rename(columns={'title':'identifier'})
arx = arx.to_dict('records')

In [ ]:
upsert(bioarx,index,create_lbert_embed,lbert_bundle,'bioarx')

  0%|          | 0/226 [00:00<?, ?it/s]

In [ ]:
upsert(plos,index,create_lbert_embed,lbert_bundle,'plos')

  0%|          | 0/134 [00:00<?, ?it/s]

In [ ]:
upsert(arx,index,create_lbert_embed,lbert_bundle,'arx')

  0%|          | 0/27 [00:00<?, ?it/s]